In [35]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas
from io import StringIO
import sympy
import scipy
from sklearn.preprocessing import normalize
from pathlib import Path

In [36]:
# Funcion para correr .cpp

TP = "../src/main"
TEST_DIR = "../tests/"

def run_test(test_name: str, niter: int, eps: float): 
    !{TP} {TEST_DIR + test_name} {niter} {eps}

def run_test_pca(test_name: str, niter: int, eps: float, cant: int): 
    !{TP} {TEST_DIR + test_name} {niter} {eps} {cant}

def read_result_autovalores(test_result: str):
    return pandas.read_csv(TEST_DIR + test_result + '_autovalores.out', names=["autovalores"])

def read_result_autovectores(test_result: str):
    return pandas.read_csv(TEST_DIR + test_result + '_autovectores.out' , names=["autovectores"])

### 1. Tests metodo de la potencia

In [ ]:
# CASO SIMETRICO
A = np.array([
  [ 7,  2,  -3],
  [ 2,  2,  -2],
  [-3, -2,  -2]
])

# GUARDO TXT
np.savetxt("../tests/simetrico.txt", A, delimiter=' ')

# CORRO METODO
run_test("simetrico.txt", 100000, 1e-21)

# CARGO AUTOVALORES Y AUTOVECTORES
l = np.loadtxt("../tests/simetrico.txt_autovalores.out", dtype='f', delimiter=' ')
v = np.loadtxt("../tests/simetrico.txt_autovectores.out", dtype='f', delimiter=' ', usecols=range(3))

print(l)
print(v)


print(A @ v[:, 0])
print(l[0] * v[:,0])

for i in range(len(A)):
    print(i)
    print(np.allclose(A @ v[:, i], l[i] * v[:,i], 1e-6))


In [ ]:
# CASO (A) * (A traspuesta)
AT = A @ A.T
TA = A.T @ A

# GUARDO TXT
np.savetxt("../tests/AT.txt", AT, delimiter=' ')
np.savetxt("../tests/TA.txt", TA, delimiter=' ')

# CORRO METODO
run_test("AT.txt", 100000, 1e-21)
run_test("TA.txt", 100000, 1e-21)

# CARGO AUTOVALORES Y AUTOVECTORES
l_AT = np.loadtxt("../tests/AT.txt_autovalores.out", dtype='f', delimiter=' ')
v_AT = np.loadtxt("../tests/AT.txt_autovectores.out", dtype='f', delimiter=' ', usecols=range(3))

l_TA = np.loadtxt("../tests/TA.txt_autovalores.out", dtype='f', delimiter=' ')
v_TA = np.loadtxt("../tests/TA.txt_autovectores.out", dtype='f', delimiter=' ', usecols=range(3))

print(l_AT)
print(l_TA)
print("Bingo :)" if np.allclose(l_AT,l_TA) else "Bongo :(")
print("Bingo :)" if np.allclose(v_AT,v_TA) else "Bongo :(")

### Ej 2 Juli

### PCA

In [37]:
paths = []
imgs = []
for path in sorted(list(Path('caras').rglob('*/*.pgm'))):
    paths.append(path)
    imgs.append(plt.imread(path)[::2,::2]/255)
X = np.stack(imgs)

In [42]:
n = X.shape[0] # Cantidad de imagenes

X = X.reshape(n, -1) # X pasa a tener dimension n x (46 x 56)

X_centered = X - X.mean(0) # X.mean(0) devuelve un arreglo con la media de cada columna.

Mx = X_centered.T@X_centered/(n-1)  # Mx = Matriz de covarianza

# Calculo autovalores con numpy
# l, V = np.linalg.eigh(Mx)

# GUARDO TXT
np.savetxt("../tests/pca.txt", Mx, fmt='%f', delimiter=' ')

# CORRO METODO
#run_test("pca.txt", 10000, 1e-5)
run_test_pca("pca.txt", 10000, 1e-16, 409)      # Con tolerancia 1e-16 converge rapido, con 1e-17 ya no

# CARGO AUTOVALORES Y AUTOVECTORES
#l = np.loadtxt("../tests/pca.txt_autovalores.out", dtype='f', delimiter=' ')
#v = np.loadtxt("../tests/pca.txt_autovectores.out", dtype='f', delimiter=' ', usecols=range(3))

#print(l)


Corriendo el programa...
autovalor: 0
frene en iteracion: 39
valor = 10.9606
autovalor: 1
frene en iteracion: 28
valor = 7.93519
autovalor: 2
frene en iteracion: 57
valor = 4.24201
autovalor: 3
frene en iteracion: 133
valor = 3.44357
autovalor: 4
frene en iteracion: 47
valor = 3.07579
autovalor: 5
frene en iteracion: 65
valor = 2.16801
autovalor: 6
frene en iteracion: 132
valor = 1.60432
autovalor: 7
frene en iteracion: 97
valor = 1.41619
autovalor: 8
frene en iteracion: 175
valor = 1.22674
autovalor: 9
frene en iteracion: 78
valor = 1.1128
autovalor: 10
frene en iteracion: 200
valor = 0.917017
autovalor: 11
frene en iteracion: 101
valor = 0.843244
autovalor: 12
frene en iteracion: 197
valor = 0.722838
autovalor: 13
frene en iteracion: 185
valor = 0.654548
autovalor: 14
frene en iteracion: 254
valor = 0.604213
autovalor: 15
frene en iteracion: 378
valor = 0.569323
autovalor: 16
frene en iteracion: 325
valor = 0.547678
autovalor: 17
frene en iteracion: 122
valor = 0.524459
autovalor: 18

In [58]:
# Comparo mis autovalores con numpy

l = np.loadtxt("../tests/pca.txt_autovalores.out", dtype='f', delimiter=' ')
v = np.loadtxt("../tests/pca.txt_autovectores.out", dtype='f', delimiter=' ', usecols=range(409))

l_np, v_np = np.linalg.eigh(Mx)
#l_np = (l_np[::-1])[:409]
#v_np = (v_np[::-1])[:,:409]

#print(np.allclose(l, l_np))
#print(np.allclose(v, v_np))

#for i in range(l.size):
    #if(not np.allclose(l[i], l_np[i])):
        #print(l[i] - l_np[i])
        #print("autovalor = {}", i)
        #print("==================")

#for i in range(l.size):
    #if(not np.allclose(v[i], v_np[i])):
        #print(v[i] - v_np[i])
        #print("autovalor = {}", i)
        #print("==================")
        
    

### =======================================================================================

### 2DPCA

In [ ]:
paths = []
imgs = []
for path in sorted(list(Path('caras').rglob('*/*.pgm'))):
    paths.append(path)
    imgs.append(plt.imread(path)[::2,::2]/255)
X = np.stack(imgs)

La variante 2DPCA 2 considera una imagen en su espacio original como una matriz A ∈ Ra×b
y le asocia un feature vector Y ∈ Ra, mediante la transformacion lineal Y = AX siendo
X ∈ Rb el vector que maximiza la dispersion de los features vectors obtenidos para todo el
conjunto de imagenes. Se puede ver que la eleccion  ́optima para el vector de proyeccion X
se corresponde con el autovector de maximo autovalor de la denominada image covariance
matrix

In [ ]:
print(f"Tengo {X.shape[0]} imagenes de tamano {X.shape[1]}x{X.shape[2]}")
# A es la imagen
imgs = X

n = imgs.shape[0] # Cantidad de imagenes

A_prom = np.zeros(shape=(imgs.shape[1], imgs.shape[2]))

for i in range (n):
    A_prom += imgs[i]

A_prom /= n


# G = image covariance matrix (quiero calcularle autovalores y autovectores)
sum = 0
for j in range (n): 
    sum += (np.transpose(imgs[j] - A_prom)) @ (imgs[j] - A_prom)
G = (1/n) * sum

# GUARDO TXT
np.savetxt("../tests/2dpca_g.txt", G, delimiter=' ')

# CORRO METODO
run_test("2dpca_g.txt", 100000, 1e-21)

# CARGO AUTOVALORES Y AUTOVECTORES
l = np.loadtxt("../tests/2dpca_g.txt_autovalores.out", dtype='f', delimiter=' ')
v = np.loadtxt("../tests/2dpca_g.txt_autovectores.out", dtype='f', delimiter=' ', usecols=range(3))

print(l)

# V = matriz de feature vectors
# U = base de autovectores ortonormales de G
# V = imgs[0] @ U


# Quiero computar los Zi = (AiX1, AiX2, ..., AiXk) de cada imagen
# donde cada AiXk representa el k-esimo feature vector de la imagen Ai



In [ ]:
# Comparo mis autovalores con numpy
l = np.loadtxt("../tests/2dpca_g.txt_autovalores.out", dtype='f', delimiter=' ')
v = np.loadtxt("../tests/2dpca_g.txt_autovectores.out", dtype='f', delimiter=' ', usecols=range(G.shape[0]))

l_np, v_np = np.linalg.eigh(G)

l_np = l_np[::-1]
v_np = v_np[::-1]

print(np.allclose(l, l_np))

### Ej 3